# **STEP 1: Data Importing and Pre-processing**
## - Import dataset and describe characteristics such as dimensions, data types, file types, and import methods used
## - Clean, wrangle, and handle missing data
## - Transform data appropriately using techniques such as aggregation, normalization, and feature construction
## - Reduce redundant data and perform need-based discretization

In [2]:
# import all packages used for the project in the first cell, use code cells for code and comments, 
#and use markdown cells for headings and descriptions

In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("house_sales.csv")
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650.0,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242.0,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2.0,1.00,770.0,10000.0,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000.0,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680.0,8080.0,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


Basic Characteristics

1. Shape of the data frame

In [5]:
print("Shape (rows, columns):", df.shape)

Shape (rows, columns): (21613, 21)


2. Defining file type:

The dataset was provided as a CSV file, which is a plain-text tabular file commonly used for structured data.

2. Data types by column

In [6]:
print("Data types:")
print(df.dtypes)

Data types:
id                 int64
date              object
price            float64
bedrooms         float64
bathrooms        float64
sqft_living      float64
sqft_lot         float64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object


3. Missing values

In [7]:
df.isna().sum()

id                  0
date                0
price               0
bedrooms         1134
bathrooms        1068
sqft_living      1110
sqft_lot         1044
floors              0
waterfront          0
view                0
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated        0
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
dtype: int64

Cleaning the data

1. Separating missing value columns

In [8]:
cols_na = ["bedrooms", "bathrooms", "sqft_living", "sqft_lot"]

df_na = df[cols_na]


2. Missing percentage in each column

In [9]:
missing_percent = (df_na.isna().sum() / len (df_na)) * 100
print ("Missing percent: \n", missing_percent)

Missing percent: 
 bedrooms       5.246842
bathrooms      4.941470
sqft_living    5.135798
sqft_lot       4.830426
dtype: float64


4. Distribution in missing value columns

In [10]:
df["bedrooms"].describe()

count    20479.000000
mean         3.372821
std          0.930711
min          0.000000
25%          3.000000
50%          3.000000
75%          4.000000
max         33.000000
Name: bedrooms, dtype: float64

In [11]:
df["bathrooms"].describe()

count    20545.000000
mean         2.113507
std          0.768913
min          0.000000
25%          1.500000
50%          2.250000
75%          2.500000
max          8.000000
Name: bathrooms, dtype: float64

5. Filling in missing values

    a. bedrooms
        This columns missing percentage is under 10% and the variable is discrete with clear central tendency. Most homes have 3 bedrooms, due to outliers, the mean would not be a reliable choice. The median is more robust to those outliers and better represents a typical value. For these reasons, the median, was used to fill the missing bedroom values. 

In [12]:
df["bedrooms"] = df["bedrooms"].fillna(df["bedrooms"].median())

    b. bathrooms

In [13]:
df["bathrooms"] = df ["bathrooms"].fillna(df["bathrooms"].median())

    c. sqft_living 
    Missing values in this columns were filled using the median for each bedroom count to keep estimates accruate. sqft_living15 was avoided becasue it represents nearby homes, not the specific property.
    
        

In [14]:
for b in sorted(df['bedrooms'].unique()):
    
    mis_sq = (df['bedrooms'] == b) & (df['sqft_living'].isna())

    median_sqft = df.loc[df['bedrooms'] == b, 'sqft_living'].median()

    df.loc[mis_sq, 'sqft_living'] = median_sqft

    d. sqft_lot
        Misisng values in sqft_lot were filled by using median lot size within each zip code. Lot size varies heavily by location, so grouping by zip code provides more realistic estimates than using one overall median. 

In [15]:
for z in sorted(df['zipcode'].unique()):
    
    zip = (df['zipcode'] == z) & (df['sqft_lot'].isna())
    
    median_lot = df.loc[df['zipcode'] == z, 'sqft_lot'].median()
    
    df.loc[zip, 'sqft_lot'] = median_lot

                                  

 6. Converting data types

    a. date
        The date column was converted to datetime to allow accurate time-based calculations and avoid treating the values as plain text, 

In [16]:
df['date'] = pd.to_datetime(df['date'])

df['date'].head()
df.dtypes

id                        int64
date             datetime64[ns]
price                   float64
bedrooms                float64
bathrooms               float64
sqft_living             float64
sqft_lot                float64
floors                  float64
waterfront                int64
view                      int64
condition                 int64
grade                     int64
sqft_above                int64
sqft_basement             int64
yr_built                  int64
yr_renovated              int64
zipcode                   int64
lat                     float64
long                    float64
sqft_living15             int64
sqft_lot15                int64
dtype: object

7. Duplicate checks and redundant data

In [17]:
df.duplicated().sum()

np.int64(0)

8. Impossible data

In [18]:
df[df['bedrooms'] < 0]

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15


In [19]:
df[df['bathrooms'] < 0]

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15


In [20]:
df[df['floors'] < 1]

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15


In [21]:
df[df['sqft_living'] <= 0]
df[df['sqft_lot'] <= 0]

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15


In [22]:
df[df['price'] <= 0]

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15


9. Outliers
    
    a. bedrooms
       A single extreme outlier was found in the bedrooms column where a property was listed with 33 bedroomsl. Based on the sq footage of the home, bathrooms and price, this was ultimatley determined to be a data entry error, and the value was corrected to 3.

In [23]:
col = 'bedrooms'

Q1 = df[col].quantile(0.25)
Q3 = df[col].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

df[(df[col] < lower) | (df[col] > upper)][['bedrooms']]


,bedrooms
154,1.0
209,6.0
232,6.0
239,6.0
264,1.0
...,...
21375,6.0
21443,1.0
21519,1.0
21522,6.0


In [24]:
df ['bedrooms'].describe()

count    21613.00000
mean         3.35326
std          0.90977
min          0.00000
25%          3.00000
50%          3.00000
75%          4.00000
max         33.00000
Name: bedrooms, dtype: float64

In [25]:
df[df['bedrooms'] > 10][['bedrooms']]

,bedrooms
15870,33.0


In [26]:
df.loc[15870]

id                        2402100895
date             2014-06-25 00:00:00
price                       640000.0
bedrooms                        33.0
bathrooms                       1.75
sqft_living                   1620.0
sqft_lot                      6000.0
floors                           1.0
waterfront                         0
view                               0
condition                          5
grade                              7
sqft_above                      1040
sqft_basement                    580
yr_built                        1947
yr_renovated                       0
zipcode                        98103
lat                          47.6878
long                        -122.331
sqft_living15                   1330
sqft_lot15                      4700
Name: 15870, dtype: object

In [27]:
df.loc[15870, 'bedrooms'] = 3

    b. bathrooms

In [28]:
df['bathrooms'].describe

<bound method NDFrame.describe of 0        1.00
1        2.25
2        1.00
3        3.00
4        2.00
         ... 
21608    2.50
21609    2.50
21610    0.75
21611    2.50
21612    0.75
Name: bathrooms, Length: 21613, dtype: float64>

In [30]:
df['bathrooms'].sort_values().head(10)
df['bathrooms'].sort_values(ascending=False).head(10)

12777    8.00
7252     8.00
9254     7.75
8546     7.50
8092     6.75
20578    6.50
21506    6.50
14556    6.25
18302    6.25
4035     6.00
Name: bathrooms, dtype: float64

    c. sqft_living

In [31]:
df['sqft_living'].describe()
df['sqft_living'].sort_values().head(20)
df['sqft_living'].sort_values(ascending=False).head(20)

7252     12050.0
3914     10040.0
8092      9640.0
4411      9200.0
14556     8670.0
18302     8020.0
1164      8010.0
1448      8000.0
13411     7880.0
19858     7850.0
16773     7730.0
18477     7710.0
14032     7620.0
4024      7480.0
12370     7440.0
11871     7420.0
10373     7400.0
1315      7390.0
2713      7350.0
18594     7320.0
Name: sqft_living, dtype: float64

    d. sqft_lot

In [32]:
df['sqft_lot'].describe()
df['sqft_lot'].sort_values().head(10)
df['sqft_lot'].sort_values(ascending=False).head(10)

1719     1651359.0
17319    1164794.0
7647     1074218.0
3949      982998.0
4441      982278.0
6691      920423.0
7077      881654.0
9714      871200.0
20452     871200.0
4540      843309.0
Name: sqft_lot, dtype: float64

    e. floors

In [33]:
df['floors'].describe()
df['floors'].sort_values().head(10)
df['floors'].sort_values(ascending=False).head(10)

18477    3.5
875      3.5
20308    3.5
11594    3.5
20772    3.5
10077    3.5
14885    3.5
15424    3.5
19757    3.0
20070    3.0
Name: floors, dtype: float64

    f. condition

In [34]:
df['condition'].describe()
df['condition'].value_counts()

condition
3    14031
4     5679
5     1701
2      172
1       30
Name: count, dtype: int64

    g. grade

In [35]:
df['grade'].describe()
df['grade'].value_counts()

grade
7     8981
8     6068
9     2615
6     2038
10    1134
11     399
5      242
12      90
4       29
13      13
3        3
1        1
Name: count, dtype: int64

    h. yr_built

In [36]:
df['yr_built'].sort_values().head(10)
df['yr_built'].sort_values(ascending=False).head(10)

21369    2015
1763     2015
21262    2015
14215    2015
20852    2015
20963    2015
14925    2015
19907    2015
21333    2015
7526     2015
Name: yr_built, dtype: int64

    i. sqft_living15

In [37]:
df['sqft_living15'].describe()
df['sqft_living15'].sort_values().head(10)
df['sqft_living15'].sort_values(ascending=False).head(10)

19858    6210
10373    6110
21540    5790
20563    5790
16430    5790
1123     5790
20830    5790
5451     5790
11871    5610
1530     5600
Name: sqft_living15, dtype: int64

    j. sqft_lot15

In [38]:
df['sqft_lot15'].describe()
df['sqft_lot15'].sort_values().head(10)
df['sqft_lot15'].sort_values(ascending=False).head(10)

9714     871200
20452    858132
13464    560617
8664     438213
3801     434728
1719     425581
19156    422967
6691     411962
21431    392040
15620    392040
Name: sqft_lot15, dtype: int64

    k. price

In [40]:
df.sort_values('price', ascending=True).head(50)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
1149,3421079032,2015-02-17,75000.0,1.0,0.00,670.0,43377.0,1.0,0,0,...,3,670,0,1966,0,98022,47.2638,-121.906,1160,42882
15293,40000362,2014-05-06,78000.0,2.0,1.00,780.0,8842.5,1.0,0,0,...,5,780,0,1942,0,98168,47.4739,-122.280,1700,10387
465,8658300340,2014-05-23,80000.0,1.0,0.75,430.0,5050.0,1.0,0,0,...,4,430,0,1912,0,98014,47.6499,-121.909,1200,7500
16198,3028200080,2015-03-24,81000.0,3.0,2.25,730.0,9975.0,1.0,0,0,...,5,730,0,1943,0,98168,47.4808,-122.315,860,9000
8274,3883800011,2014-11-05,82000.0,3.0,1.00,860.0,10426.0,1.0,0,0,...,6,860,0,1954,0,98146,47.4987,-122.341,1140,11250
2141,1623049041,2014-05-08,82500.0,2.0,1.00,520.0,22334.0,1.0,0,0,...,5,520,0,1951,0,98168,47.4799,-122.296,1572,10570
18468,7999600180,2014-05-29,83000.0,2.0,1.00,900.0,8580.0,1.0,0,0,...,5,900,0,1918,0,98168,47.4727,-122.270,2060,6533
3767,1523049188,2015-04-30,84000.0,2.0,1.00,700.0,20130.0,1.0,0,0,...,6,700,0,1949,0,98168,47.4752,-122.271,1490,18630
16714,1322049150,2015-03-05,85000.0,2.0,1.00,910.0,9753.0,1.0,0,0,...,5,910,0,1947,0,98032,47.3897,-122.236,1160,7405
10253,2422049104,2014-09-15,85000.0,2.0,1.00,830.0,9000.0,1.0,0,0,...,6,830,0,1939,0,98032,47.3813,-122.243,1160,7680


In [41]:
df.sort_values('price', ascending=False).head(10)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
7252,6762700020,2014-10-13,7700000.0,6.0,8.00,12050.0,27600.0,2.5,0,3,...,13,8570,3480,1910,1987,98102,47.6298,-122.323,3940,8800
3914,9808700762,2014-06-11,7062500.0,5.0,4.50,10040.0,37325.0,2.0,1,2,...,11,7680,2360,1940,2001,98004,47.6500,-122.214,3930,25449
9254,9208900037,2014-09-19,6885000.0,6.0,7.75,2940.0,31374.0,2.0,0,4,...,13,8860,1030,2001,0,98039,47.6305,-122.240,4540,42730
4411,2470100110,2014-08-04,5570000.0,5.0,5.75,9200.0,16818.5,2.0,0,0,...,13,6200,3000,2001,0,98039,47.6289,-122.233,3560,24345
1448,8907500070,2015-04-13,5350000.0,5.0,5.00,8000.0,23985.0,2.0,0,4,...,12,6720,1280,2009,0,98004,47.6232,-122.220,4600,21750
1315,7558700030,2015-04-13,5300000.0,6.0,6.00,7390.0,24829.0,2.0,1,4,...,12,5000,2390,1991,0,98040,47.5631,-122.210,4320,24619
1164,1247600105,2014-10-20,5110800.0,5.0,5.25,8010.0,45517.0,2.0,1,4,...,12,5990,2020,1999,0,98033,47.6767,-122.211,3430,26788
8092,1924059029,2014-06-17,4668000.0,5.0,6.75,9640.0,13068.0,1.0,1,4,...,12,4820,4820,1983,2009,98040,47.5570,-122.210,3270,10454
2626,7738500731,2014-08-15,4500000.0,5.0,5.50,6640.0,40014.0,2.0,1,4,...,12,6350,290,2004,0,98155,47.7493,-122.280,3030,23408
8638,3835500195,2014-06-18,4489000.0,4.0,3.00,6430.0,27517.0,2.0,0,0,...,12,6430,0,2001,0,98004,47.6208,-122.219,3720,14592


10. Validation

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             21613 non-null  int64         
 1   date           21613 non-null  datetime64[ns]
 2   price          21613 non-null  float64       
 3   bedrooms       21613 non-null  float64       
 4   bathrooms      21613 non-null  float64       
 5   sqft_living    21613 non-null  float64       
 6   sqft_lot       21613 non-null  float64       
 7   floors         21613 non-null  float64       
 8   waterfront     21613 non-null  int64         
 9   view           21613 non-null  int64         
 10  condition      21613 non-null  int64         
 11  grade          21613 non-null  int64         
 12  sqft_above     21613 non-null  int64         
 13  sqft_basement  21613 non-null  int64         
 14  yr_built       21613 non-null  int64         
 15  yr_renovated   2161

In [44]:
df.describe()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.161300e+04,21613,2.161300e+04,21613.000000,21613.000000,21613.000000,2.161300e+04,21613.000000,21613.000000,21613.000000,...,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000
mean,4.580302e+09,2014-10-29 04:38:01.959931648,5.400881e+05,3.351872,2.120252,2072.164438,1.485375e+04,1.494309,0.007542,0.234303,...,7.656873,1788.390691,291.509045,1971.005136,84.402258,98077.939805,47.560053,-122.213896,1986.552492,12768.455652
min,1.000102e+06,2014-05-02 00:00:00,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,0.000000,...,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123049e+09,2014-07-22 00:00:00,3.219500e+05,3.000000,1.750000,1440.000000,5.080000e+03,1.000000,0.000000,0.000000,...,7.000000,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.471000,-122.328000,1490.000000,5100.000000
50%,3.904930e+09,2014-10-16 00:00:00,4.500000e+05,3.000000,2.250000,1900.000000,7.663000e+03,1.500000,0.000000,0.000000,...,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.571800,-122.230000,1840.000000,7620.000000
75%,7.308900e+09,2015-02-17 00:00:00,6.450000e+05,4.000000,2.500000,2520.000000,1.050000e+04,2.000000,0.000000,0.000000,...,8.000000,2210.000000,560.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,9.900000e+09,2015-05-27 00:00:00,7.700000e+06,10.000000,8.000000,12050.000000,1.651359e+06,3.500000,1.000000,4.000000,...,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000
std,2.876566e+09,NaN,3.671272e+05,0.887140,0.750257,899.270450,4.051167e+04,0.539989,0.086517,0.766318,...,1.175459,828.090978,442.575043,29.373411,401.679240,53.505026,0.138564,0.140828,685.391304,27304.179631


In [45]:
df.isna().sum()

id               0
date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64

# **STEP 2: Data Analysis and Visualization**
## -Identify categorical, ordinal, and numerical variables within the data
## -Provide measures of centrality and distribution with visualizations
## -Diagnose for correlations between variables and determine independent and dependent variables
## -Perform exploratory analysis in combination with visualization techniques to discover patterns and features of interest

# **STEP 3: Data Analytics**
## -Determine the need for a supervised or unsupervised learning method and identify dependent and independent variables
## -Train, test, and provide accuracy and evaluation metrics for model results
